In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
# IMPORTAMOS LOS DF
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

In [18]:
# COLUMNAS CON CARACTERÍSTICAS DEL LIBRO
tfidf_columns = [col for col in train.columns if col not in genre_columns]
tfidf_columns = [col for col in tfidf_columns if col.isalpha()]
genre_columns = [col for col in train.columns if col.startswith('genre_')]

similarity_data = train[tfidf_columns + genre_columns]

# BOOLEANOS A INT
boolean_columns = similarity_data.select_dtypes(include=['bool']).columns.tolist()
similarity_data[boolean_columns] = similarity_data[boolean_columns].astype(int)

similarity_data = similarity_data.drop(columns=['reviewer'], errors='ignore')

print(similarity_data.dtypes)


rating                     float64
about                      float64
all                        float64
amazing                    float64
an                         float64
                            ...   
genre_Self Help              int64
genre_Self-improvement       int64
genre_Spiritual Warfare      int64
genre_Spirituality           int64
genre_Thriller               int64
Length: 268, dtype: object


C:\Users\Sandra\AppData\Local\Temp\ipykernel_8632\306941180.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  similarity_data[boolean_columns] = similarity_data[boolean_columns].astype(int)


In [19]:
# SIMILITUD DE COSENO ENTRE LIBROS
cosine_sim = cosine_similarity(similarity_data)
print(cosine_sim[:5, :5])

[[ 1.          0.59040327  0.59040327 -0.23206964 -0.23206964]
 [ 0.59040327  1.          0.59040327 -0.23206964 -0.23206964]
 [ 0.59040327  0.59040327  1.         -0.23206964 -0.23206964]
 [-0.23206964 -0.23206964 -0.23206964  1.          0.76791622]
 [-0.23206964 -0.23206964 -0.23206964  0.76791622  1.        ]]


In [31]:
# FUNCIÓN PARA RECOMENDAR LIBROS
def recommend_books(book_title, train_df, cosine_sim_matrix, num_recommendations=5):
    
    book_index = train_df[train_df['book_title'] == book_title].index[0]
    
    # OBTIENE PUNTUACIONES SIMILARES
    sim_scores = list(enumerate(cosine_sim_matrix[book_index]))
    
    # ORDENA DE MAYOR A MENOR
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # EVITA DUPLICADOS EL SET
    unique_recommendations = set()
    
    for i in range(1, len(sim_scores)):
        book_idx = sim_scores[i][0]  
        recommended_title = train_df['book_title'].iloc[book_idx]  

        if recommended_title != book_title and recommended_title not in unique_recommendations:
            unique_recommendations.add(recommended_title)
            
        if len(unique_recommendations) >= num_recommendations:
            break

    return list(unique_recommendations)[:num_recommendations]

# EJEMPLO
book_to_recommend = 'Just Because'
if book_to_recommend in train['book_title'].values:
    recommendations = recommend_books(book_to_recommend, train, cosine_sim, num_recommendations=5)
    print("Libros recomendados:")
    print(recommendations)
else:
    print(f"El libro '{book_to_recommend}' no se encontró en el DataFrame.")

Libros recomendados:
['The Psychology of Money: Timeless lessons on wealth, greed, and happiness', 'I Am Thankful: A Thanksgiving Book for Kids', 'Llama Llama Gives Thanks', 'The Coworker', 'If Animals Kissed Good Night']


In [34]:
recommended_books = [
    'The Psychology of Money: Timeless lessons on wealth, greed, and happiness',
    'I Am Thankful: A Thanksgiving Book for Kids',
    'Llama Llama Gives Thanks',
    'The Coworker',
    'If Animals Kissed Good Night'
]

# Filtrar el DataFrame para encontrar los géneros de los libros recomendados
genres_list = []

for book in recommended_books:
    # Filtrar el DataFrame para el libro actual
    book_genres = train[train['book_title'] == book]
    
    # Obtener los géneros
    if not book_genres.empty:
        # Extraer géneros de las columnas de género (ejemplo: genre_Self Help, genre_Fiction, etc.)
        genres = book_genres.filter(like='genre_').any().index[book_genres.filter(like='genre_').any()].tolist()
        genres_list.append((book, genres))

# Mostrar los géneros de cada libro recomendado
for book, genres in genres_list:
    print(f'Libro: "{book}" - Géneros: {genres}')

Libro: "The Psychology of Money: Timeless lessons on wealth, greed, and happiness" - Géneros: ['genre_ Childrens', 'genre_ Holiday', 'genre_Picture Books']
Libro: "I Am Thankful: A Thanksgiving Book for Kids" - Géneros: ['genre_ Childrens', 'genre_Picture Books']
Libro: "Llama Llama Gives Thanks" - Géneros: ['genre_ Animals', 'genre_ Childrens', 'genre_ Holiday', 'genre_ Kids Fiction', 'genre_Picture Books']
Libro: "The Coworker" - Géneros: ['genre_ Mystery', 'genre_Thriller']
Libro: "If Animals Kissed Good Night" - Géneros: ['genre_ Animals', 'genre_ Childrens', 'genre_ Family', 'genre_ Fiction', 'genre_ Poetry', 'genre_ Storytime', 'genre_Picture Books']
